- try transfer learning (consider only fold 5 file)

In [1]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from umap import UMAP
from sklearn import preprocessing
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import KMeans
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold

sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torch.nn.modules.loss import _WeightedLoss

In [2]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [3]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [4]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [5]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
test = test[test.index.isin(cons_test_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)
non_targets = non_targets[non_targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

In [6]:
non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
nontarget_dists = pd.DataFrame(np.sum(non_targets[non_target_feats])).reset_index(drop=False)
nontarget_dists.columns = ["target", "number"]
nontarget_dists = nontarget_dists.sort_values("number", ascending=False).reset_index(drop=True)
drop_list1 = list(nontarget_dists[nontarget_dists.number==0]["target"].values)
print("first drop", len(drop_list1))
non_targets.drop(drop_list1, axis=1, inplace=True)
print("shape after 1st drop:", non_targets.shape)
non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
print(len(non_target_feats))

first drop 71
shape after 1st drop: (21948, 332)
331


In [7]:
all_targets = pd.concat([targets, non_targets], axis=1)

# Feature engineering 

In [8]:
X = train.iloc[:,4:].copy().values
select = VarianceThreshold(threshold=0.7)
X_new = select.fit_transform(X)
drop_feats = list(np.array(train.iloc[:,4:].columns)[select.get_support()==False])
len(drop_feats)

train.drop(drop_feats, axis=1, inplace=True)
test.drop(drop_feats, axis=1, inplace=True)

In [9]:
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [10]:
# rank gauss
for i in c_feats + g_feats:
    ss = preprocessing.QuantileTransformer(n_quantiles=1000, random_state=0, output_distribution="normal")
    ss.fit(train[i].values.reshape(-1,1))
    train[i] = ss.transform(train[i].values.reshape(-1,1))
    test[i] = ss.transform(test[i].values.reshape(-1,1))

In [11]:
c_num = 10
pca_c_cols = ["pca-c"+str(i+1) for i in range(c_num)]
pca = PCA(n_components=c_num,random_state=42)
c_train = pca.fit_transform(train[c_feats])
c_test = pca.transform(test[c_feats])
c_train = pd.DataFrame(c_train, columns=pca_c_cols)
c_test = pd.DataFrame(c_test, columns=pca_c_cols)

g_num = 60
pca_g_cols = ["pca-g"+str(i+1) for i in range(g_num)]
pca = PCA(n_components=g_num, random_state=42)
g_train = pca.fit_transform(train[g_feats])
g_test = pca.transform(test[g_feats])
g_train = pd.DataFrame(g_train, columns=pca_g_cols)
g_test = pd.DataFrame(g_test, columns=pca_g_cols)

train = pd.concat([train, c_train],axis=1)
test = pd.concat([test, c_test],axis=1)
train = pd.concat([train, g_train],axis=1)
test = pd.concat([test, g_test],axis=1)

In [12]:
#uc_num = 1
#um = UMAP(n_neighbors=30, n_components=uc_num, random_state=42)
#um_c_cols = ["um-c"+str(i+1) for i in range(uc_num)]
#uc_train = um.fit_transform(train[c_feats])
#uc_test = um.transform(test[c_feats])
#uc_train = pd.DataFrame(uc_train, columns=um_c_cols)
#uc_test = pd.DataFrame(uc_test, columns=um_c_cols)

#ug_num = 5
#um = UMAP(n_neighbors=30, n_components=ug_num, random_state=42)
#um_g_cols = ["um-g"+str(i+1) for i in range(ug_num)]
#ug_train = um.fit_transform(train[g_feats])
#ug_test = um.transform(test[g_feats])
#ug_train = pd.DataFrame(ug_train, columns=um_g_cols)
#ug_test = pd.DataFrame(ug_test, columns=um_g_cols)

#train = pd.concat([train, uc_train],axis=1)
#test = pd.concat([test, uc_test],axis=1)
#train = pd.concat([train, ug_train],axis=1)
#test = pd.concat([test, ug_test],axis=1)

In [13]:
def fe(df):
    tmp = df.copy()
    tmp = pd.get_dummies(tmp, columns=['cp_time','cp_dose'])
    tmp.drop(["cp_type", "sig_id"], axis=1, inplace=True) 
    return tmp

train = fe(train)
test = fe(test)

print(train.shape, test.shape)

(21948, 914) (3624, 914)


In [14]:
fn_train = train.copy().to_numpy()
fn_test = test.copy().to_numpy()

fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()
fn_all_targets = all_targets.drop("sig_id", axis=1).copy().to_numpy()

# modelling

In [15]:
class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets, n_classes, smoothing=0.0):
        assert 0 <= smoothing <= 1
        with torch.no_grad():
            targets = targets * (1 - smoothing) + torch.ones_like(targets).to(device) * smoothing / n_classes
        return targets

    def forward(self, inputs, targets):
        targets = SmoothCrossEntropyLoss()._smooth(targets, inputs.shape[1], self.smoothing)

        if self.weight is not None:
            inputs = inputs * self.weight.unsqueeze(0)

        loss = F.binary_cross_entropy_with_logits(inputs, targets)

        return loss

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)
def mean_log_loss(y_true, y_pred):
    metrics = []
    for i, target in enumerate(target_feats):
        metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
    return np.mean(metrics)

def seed_everything(seed=42): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class MoaModel(nn.Module):
    def __init__(self, num_columns, last_num):
        super(MoaModel, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_columns)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_columns, 1024))
        self.relu1 = nn.LeakyReLU()
        
        self.batch_norm2 = nn.BatchNorm1d(1024)
        self.dropout2 = nn.Dropout(0.1)
        self.dense2 = nn.utils.weight_norm(nn.Linear(1024, 1024))
        self.relu2 = nn.LeakyReLU()
        
        self.batch_norm3 = nn.BatchNorm1d(1024)
        self.dropout3 = nn.Dropout(0.1)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1024, last_num))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = self.relu1(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = self.relu2(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

cuda


# modelling

In [17]:
batch_size = 128
n_folds=5
EARLY_STOPPING_STEPS = 10
smoothing = 0.001
p_min = smoothing
p_max = 1 - smoothing

def torch_tl(tr, target, te, sample_seed, init_num, files):
    seed_everything(seed=sample_seed) 
    X_train = tr.copy()
    y_train = target.copy()
    X_test = te.copy()
    test_len = X_test.shape[0]
    
    if files == []:
        train_epochs = 10
        mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=224)
        ans = []
    else:
        train_epochs = 20
        mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=2)
        
    metric = lambda inputs, targets : F.binary_cross_entropy((torch.clamp(torch.sigmoid(inputs), p_min, p_max)), targets)
    models = []
    
    X_test = torch.tensor(X_test, dtype=torch.float32)
    X_test = torch.utils.data.TensorDataset(X_test) 
    test_loader = torch.utils.data.DataLoader(X_test, batch_size=batch_size, shuffle=False)
    
    oof = np.zeros([len(X_train),y_train.shape[1]])
    oof_targets = np.zeros([len(X_train),y_train.shape[1]])
    pred_value = np.zeros([test_len, y_train.shape[1]])
    scores = []
    for fold, (train_index, valid_index) in enumerate(mskf.split(X_train, y_train)):
        print("Fold "+str(fold+1))
        X_train2 = torch.tensor(X_train[train_index,:], dtype=torch.float32)
        y_train2 = torch.tensor(y_train[train_index], dtype=torch.float32)

        X_valid2 = torch.tensor(X_train[valid_index,:], dtype=torch.float32)
        y_valid2 = torch.tensor(y_train[valid_index], dtype=torch.float32)
        
        train = torch.utils.data.TensorDataset(X_train2, y_train2)
        valid = torch.utils.data.TensorDataset(X_valid2, y_valid2)
                
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) 
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
            
        clf = MoaModel(init_num, len(non_target_feats)+len(target_feats))

        loss_fn = SmoothCrossEntropyLoss(smoothing=smoothing)
        if files != []:
            clf.load_state_dict(torch.load(files[0]))
            for param in clf.parameters():
                param.requires_grad = False
            # reinitialze
            clf.dense3 = nn.utils.weight_norm(nn.Linear(1024, len(target_feats)))
            for param in clf.dense3.parameters():
                param.requires_grad = True
            optimizer = optim.Adam(clf.parameters(), lr = 0.001, weight_decay=1e-5) 
            scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=train_epochs, steps_per_epoch=len(train_loader))
        else:
            optimizer = optim.Adam(clf.parameters(), lr = 0.001, weight_decay=1e-5) 
            scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=train_epochs, steps_per_epoch=len(train_loader))
        
        clf.to(device)
        best_val_loss = np.inf
        stop_counts = 0
        
        for epoch in range(train_epochs):
            start_time = time.time()
            clf.train()
            avg_loss = 0.
            sm_avg_loss = 0.
            for x_batch, y_batch in tqdm(train_loader, disable=True):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch) 
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()
                avg_loss += loss.item() / len(train_loader)  
                sm_avg_loss += metric(y_pred, y_batch) / len(train_loader)      
            
            clf.eval()
            avg_val_loss = 0.
            sm_avg_val_loss = 0.

            for i, (x_batch, y_batch) in enumerate(valid_loader): 
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch).detach()
                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
                sm_avg_val_loss += metric(y_pred, y_batch) / len(valid_loader)
                
            elapsed_time = time.time() - start_time 
                    
            if sm_avg_val_loss < best_val_loss:
                stop_counts = 0
                best_val_loss = sm_avg_val_loss
                print('Best: Epoch {} \t loss={:.5f} \t val_loss={:.5f} \t sm_loss={:.5f} \t sm_val_loss={:.5f} \t time={:.2f}s'.format(
                    epoch + 1, avg_loss, avg_val_loss, sm_avg_loss, sm_avg_val_loss, elapsed_time))
                if files == []:
                    torch.save(clf.state_dict(), 'parameters'+str(fold+1)+'.pt')
                else:    
                    torch.save(clf.state_dict(), 'best-model-parameters.pt')
            else:
                stop_counts += 1
           
        if files == []:
            pred_model = MoaModel(init_num, len(non_target_feats)+len(target_feats))
            pred_model.load_state_dict(torch.load('parameters'+str(fold+1)+'.pt'))
            if fold == 4:
                ans.append('parameters.pt')
        else:
            pred_model = MoaModel(init_num, len(target_feats))
            pred_model.load_state_dict(torch.load('best-model-parameters.pt'))         
        pred_model.eval()
        
        # validation check ----------------
        oof_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        target_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        for i, (x_batch, y_batch) in enumerate(valid_loader): 
                y_pred = pred_model(x_batch).detach()
                oof_epoch[i * batch_size:(i+1) * batch_size,:] = torch.clamp(torch.sigmoid(y_pred.cpu()), p_min, p_max)
                target_epoch[i * batch_size:(i+1) * batch_size,:] = y_batch.cpu().numpy()
        print("Fold {} log loss: {}".format(fold+1, mean_log_loss(target_epoch, oof_epoch)))
        scores.append(mean_log_loss(target_epoch, oof_epoch))
        oof[valid_index,:] = oof_epoch
        oof_targets[valid_index,:] = target_epoch
        #-----------------------------------
        
        if files != []:
            # test predcition --------------
            test_preds = np.zeros([test_len, y_train.shape[1]])
            for i, (x_batch,) in enumerate(test_loader): 
                y_pred = pred_model(x_batch).detach()
                test_preds[i * batch_size:(i+1) * batch_size, :] = torch.clamp(torch.sigmoid(y_pred.cpu()), p_min, p_max)
            pred_value += test_preds / n_folds
            # ------------------------------
        
    print("Seed {}".format(sample_seed))
    for i, ele in enumerate(scores):
        print("Fold {} log loss: {}".format(i+1, scores[i]))
    print("Std of log loss: {}".format(np.std(scores)))
    print("Total log loss: {}".format(mean_log_loss(oof_targets, oof)))
    
    if files != []:
        return oof, oof_targets, pred_value
    else:
        return ans

In [18]:
files = torch_tl(fn_train, fn_all_targets, fn_test, 0, fn_train.shape[1], [])
print(len(files))
seeds = [0,1,2,3] 
target_oof = np.zeros([len(fn_train),fn_targets.shape[1]])
target_pred = np.zeros([len(fn_test),fn_targets.shape[1]])
    
for seed_ in seeds:
    oof, oof_targets, pytorch_pred = torch_tl(fn_train, fn_targets, fn_test, seed_, fn_train.shape[1], files)
    target_oof += oof / len(seeds)
    target_pred += pytorch_pred / len(seeds)

print("Total log loss in targets: {}".format(mean_log_loss(oof_targets, target_oof)))

Fold 1
Best: Epoch 1 	 loss=0.28681 	 val_loss=0.01191 	 sm_loss=0.28679 	 sm_val_loss=0.01189 	 time=1.47s
Best: Epoch 2 	 loss=0.01107 	 val_loss=0.01091 	 sm_loss=0.01110 	 sm_val_loss=0.01086 	 time=0.91s
Best: Epoch 3 	 loss=0.01058 	 val_loss=0.01026 	 sm_loss=0.01068 	 sm_val_loss=0.01041 	 time=0.90s
Best: Epoch 5 	 loss=0.01020 	 val_loss=0.01014 	 sm_loss=0.01036 	 sm_val_loss=0.01032 	 time=1.14s
Best: Epoch 6 	 loss=0.01007 	 val_loss=0.01001 	 sm_loss=0.01024 	 sm_val_loss=0.01020 	 time=0.90s
Best: Epoch 7 	 loss=0.00994 	 val_loss=0.00985 	 sm_loss=0.01011 	 sm_val_loss=0.01004 	 time=0.93s
Best: Epoch 8 	 loss=0.00973 	 val_loss=0.00966 	 sm_loss=0.00991 	 sm_val_loss=0.00985 	 time=0.91s
Best: Epoch 9 	 loss=0.00944 	 val_loss=0.00950 	 sm_loss=0.00963 	 sm_val_loss=0.00969 	 time=0.93s
Best: Epoch 10 	 loss=0.00920 	 val_loss=0.00945 	 sm_loss=0.00941 	 sm_val_loss=0.00964 	 time=0.90s
Fold 1 log loss: 0.01662170138916034
Fold 2
Best: Epoch 1 	 loss=0.28617 	 val_loss

FileNotFoundError: [Errno 2] No such file or directory: 'parameters.pt'

In [19]:
aucs = []
for task_id in range(targets.shape[1]-1):
    aucs.append(roc_auc_score(y_true=targets.iloc[:, task_id+1].values,
                              y_score=target_oof[:, task_id]))
print(f"Overall AUC : {np.mean(aucs)}")

Overall AUC : 0.5


In [20]:
t = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
train_checkscore = t.copy()
train_checkscore.loc[train_checkscore.index.isin(cons_train_index),target_feats] = target_oof
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0
t.drop("sig_id", axis=1, inplace=True)
print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.array(train_checkscore.iloc[:,1:]))))

OOF log loss:  0.11859124020134244


In [21]:
sub.loc[cons_test_index,target_feats] = target_pred
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)